In [55]:
#Sorry for this crazy import cell... this will be cleaned up

import psycopg2
import numpy
import nltk
import unicodedata
import pandas as pd
import pprint
import pickle
import re
import os
from nltk.corpus import wordnet
import time
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import nltk.corpus
from nltk.text import TextCollection
import sklearn
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
import numpy
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier


stops = nltk.corpus.stopwords.words('english')
wnl = nltk.WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mark\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

In [2]:
#Input your PostGres credentials to connect

dbname = 'yelp'
username = 'postgres'
host = 'localhost'
password = 'Khoobam1234!'

conn = psycopg2.connect('dbname={} user={} host={} password={}'.format(dbname, username, host, password))
cur = conn.cursor()

In [3]:
cur = conn.cursor()
cur.execute("""
    SELECT * FROM review LIMIT 1000
""")

cols = ['review_id', 'user_id', 'business_id', 'stars', 'review_date', 'review_text', 'useful', 'funny', 'cool']

review_sample = pd.DataFrame(cur.fetchall(), columns=cols)



In [ ]:
review_sample

In [ ]:
review_sample['review_text']

Contractions:
https://gist.github.com/J3RN/ed7b420a6ea1d5bd6d06

In [4]:
common_nonneg_contr = ["could've",
"he'd",
"he'd've",
"he'll",
"he's",
"how'd",
"how'll",
"how's",
"i'd",
"i'd've",
"i'll",
"i'm",
"i've",
"it'd",
"it'd've",
"it'll",
"it's",
"let's",
"ma'am",
"might've",
"must've",
"o'clock",
"'ow's'at",
"she'd",
"she'd've",
"she'll",
"she's",
"should've",
"somebody'd",
"somebody'd've",
"somebody'll",
"somebody's",
"someone'd",
"someone'd've",
"someone'll",
"someone's",
"something'd",
"something'd've",
"something'll",
"something's",
"that'll",
"that's",
"there'd",
"there'd've",
"there're",
"there's",
"they'd",
"they'd've",
"they'll",
"they're",
"they've",
"'twas",
"we'd",
"we'd've",
"we'll",
"we're",
"we've",
"what'll",
"what're",
"what's",
"what've",
"when's",
"where'd",
"where's",
"where've",
"who'd",
"who'd've",
"who'll",
"who're",
"who's",
"who've",
"why'll",
"why're",
"why's",
"would've",
"y'all",
"y'all'll",
"y'all'd've",
"you'd",
"you'd've",
"you'll",
"you're",
"you've"]

In [5]:
neg_stops1 = ['no',
 'nor',
 'not',
 'don',
 "don't",
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'but',
 "don'",
 "ain't"]

words_to_be_added = ['us']
stops1 = ['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don't",
 'should',
 "should've",
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't"]


In [6]:
for x in neg_stops1:
    if x in stops1:
        stops1.remove(x)
        
stops1

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few

In [7]:
new_stops = stops1 + common_nonneg_contr
list(set(new_stops))

["it'd've",
 'against',
 "y'all'll",
 'as',
 "that's",
 "y'all'd've",
 'the',
 "there'd",
 'again',
 'which',
 'who',
 'do',
 'ours',
 "i'll",
 'its',
 "something'd",
 'why',
 'all',
 "they're",
 're',
 'am',
 'below',
 'off',
 "what're",
 'my',
 'should',
 'during',
 'will',
 'over',
 'same',
 "when's",
 'theirs',
 "i'd've",
 'other',
 's',
 'those',
 'her',
 'and',
 'from',
 "something'd've",
 'for',
 "somebody'd",
 'he',
 'myself',
 'until',
 "he'd've",
 "she'd've",
 "somebody's",
 'very',
 'll',
 'above',
 'a',
 'were',
 'such',
 "where's",
 "who'd",
 'then',
 'that',
 "why's",
 "they'd've",
 'here',
 'itself',
 'themselves',
 'are',
 'few',
 "you'd've",
 "there're",
 'she',
 "where've",
 "might've",
 'or',
 'them',
 "someone'd've",
 'ourselves',
 "you've",
 'of',
 'in',
 'because',
 'doing',
 'ma',
 'me',
 'on',
 "who'll",
 'does',
 'out',
 'between',
 "'ow's'at",
 'whom',
 "they'd",
 'to',
 'up',
 "something'll",
 'where',
 'both',
 'any',
 've',
 "it'll",
 'these',
 "let's",
 't

# Cell below is preproccessing and tokenezation

In [8]:
def _remove_stops(tokens):
    stops = nltk.corpus.stopwords.words('english')
    neg_stops = [
    'no', 'not', 'nor', 'don\'', 'don\'t', 'ain', 
    'ain\'t', 'aren\'t', 'aren', 'couldn', 'couldn\'t', 
    'didn', 'didn\'t', 'doesn', 'doesn\'t', 'hadn', 
    'hadn\'t', 'hasn', 'hasn\'t', 'haven', 'haven\'t',
    'isn', 'isn\'t', 'mightn', 'mightn\'t', 'mustn', 
    'mustn\'t', 'needn', 'needn\'t', 'shan', 'shan\'t',
    'shouldn', 'shouldn\'t', 'wasn', 'wasn\'t', 'weren',
    'weren\'t', 'won', 'won\'t', 'wouldn', 'wouldn\'t'
    ]
#still leaves in but and don.. fix this.. 
#doesn't get rid of other obvious stopwords, like i'm, they're....
    for x in neg_stops:
        if x in stops:
            stops.remove(x)
            
    tokens_without_stops = [token for token in tokens if token not in stops]
    return tokens_without_stops       
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def _clean_review2(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s']", '', text)  
    tokens = [token for token in text.split() if token not in new_stops]
    for i, token in enumerate(tokens):
        tokens[i] = wnl.lemmatize(token, pos= get_wordnet_pos(token))
    return tokens

def _process_review2(text):
    tokens = _remove_stops(_clean_review2(text))
    return tokens







def _process_review(text):
    def _create_stop_words():
        stops = nltk.corpus.stopwords.words('english')
    
        neg_stops = ['no',
         'nor',
         'not',
         'don',
         "don't",
         'ain',
         'aren',
         "aren't",
         'couldn',
         "couldn't",
         'didn',
         "didn't",
         'doesn',
         "doesn't",
         'hadn',
         "hadn't",
         'hasn',
         "hasn't",
         'haven',
         "haven't",
         'isn',
         "isn't",
         'mightn',
         "mightn't",
         'mustn',
         "mustn't",
         'needn',
         "needn't",
         'shan',
         "shan't",
         'shouldn',
         "shouldn't",
         'wasn',
         "wasn't",
         'weren',
         "weren't",
         "won'",
         "won't",
         'wouldn',
         "wouldn't",
         'but',
         "don'",
         "ain't"]

        common_nonneg_contr = ["could've",
        "he'd",
        "he'd've",
        "he'll",
        "he's",
        "how'd",
        "how'll",
        "how's",
        "i'd",
        "i'd've",
        "i'll",
        "i'm",
        "i've",
        "it'd",
        "it'd've",
        "it'll",
        "it's",
        "let's",
        "ma'am",
        "might've",
        "must've",
        "o'clock",
        "'ow's'at",
        "she'd",
        "she'd've",
        "she'll",
        "she's",
        "should've",
        "somebody'd",
        "somebody'd've",
        "somebody'll",
        "somebody's",
        "someone'd",
        "someone'd've",
        "someone'll",
        "someone's",
        "something'd",
        "something'd've",
        "something'll",
        "something's",
        "that'll",
        "that's",
        "there'd",
        "there'd've",
        "there're",
        "there's",
        "they'd",
        "they'd've",
        "they'll",
        "they're",
        "they've",
        "'twas",
        "we'd",
        "we'd've",
        "we'll",
        "we're",
        "we've",
        "what'll",
        "what're",
        "what's",
        "what've",
        "when's",
        "where'd",
        "where's",
        "where've",
        "who'd",
        "who'd've",
        "who'll",
        "who're",
        "who's",
        "who've",
        "why'll",
        "why're",
        "why's",
        "would've",
        "y'all",
        "y'all'll",
        "y'all'd've",
        "you'd",
        "you'd've",
        "you'll",
        "you're",
        "you've"]

        letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
          'u', 'v', 'w', 'x', 'y', 'z']
        
        ranks = ['st', 'nd', 'rd', 'th']
        
        for x in neg_stops:
            if x in stops:
                stops.remove(x)

        new_stops = stops + common_nonneg_contr + letters + ranks + [""] + ['us'] + ['']
        stops = list(set(new_stops))
        return stops

    def get_wordnet_pos(word):
        tag = nltk.pos_tag([word])[0][1][0].lower()
        tag_dict = {"a": wordnet.ADJ,
                    "n": wordnet.NOUN,
                    "v": wordnet.VERB,
                    "r": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)

    def _clean_review(text):
        text = text.lower()
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
        tokenizer = nltk.RegexpTokenizer('\w+\'?\w+')
        filtered_tokens = [(re.sub(r"[^A-Za-z\s']", '', token)) for token in tokenizer.tokenize(text)]
        stops = _create_stop_words()
        tokens = [token for token in filtered_tokens if token not in stops]
        tokens = [re.sub("'s", '', token) for token in tokens if re.sub("'s", '', token) != '']
        for i, token in enumerate(tokens):
            tokens[i] = wnl.lemmatize(token, pos= get_wordnet_pos(token))
        tokens = [token for token in tokens if token != '' and token not in stops]
        return tokens
    
    return _clean_review(text)


In [9]:
#fyi, tweaked Alice's code here to allow map_on_column to accept another function
def map_on_column(data, function):
    data['review_text'] = data['review_text'].map(lambda x: function(x))
    return data

In [10]:
map_on_column(review_sample, _process_review)

,review_id,user_id,business_id,stars,review_date,review_text,useful,funny,cool
0,fomc2nZfjK6Se9v-Ba5oWQ,bBQcAreoMmfd8LICFGWjtw,coHUUrQSdWpiFCfF2ayFNA,5,2016-01-17,"[church, like, big, beautiful, oak, tree, many...",2,0,0
1,seHlbqQhMNqCQQPb7drsRQ,D_RjBQlUppSLIJhSgvf1xg,CGpC_L37PBnIbs8TsuBEpA,5,2014-09-21,"[trek, great, line, quality, bike, entry, leve...",0,0,0
2,SwIV2t-U0QhF4GnHmPXcWw,P34rd7qs2LhNa1K0VYuVJA,wJVW-Bgc9j2aaMnwI7TH3Q,4,2011-04-22,"[experienced, charcut, alleyburger, don't, kno...",0,0,0
3,eJXmg_cHbofsjKwgcSpPXw,iLZ8eMMafDrNnRqI5164-Q,thlAnPN1ApoNxSnok_fcvA,4,2017-03-27,"[first, time, restaurant, want, try, base, rec...",0,0,0
4,tI2xPNMV9N2Z6ucMPV6vUQ,lkS0tx-RWnzvBQ1lipatXQ,CrIWqmuO2uQWwl3z11K_BA,4,2015-10-08,"[like, pizza, told, friend, place, nothing, fa...",1,0,0
5,sxWZklgHXb7ovVndf1iNew,Hh3_5hP9KsPvtn6eDHPNPw,k0JaQ2w3NIgUweASFor0lg,4,2012-07-16,"[relaxed, fun, patient, great, experience, kid...",2,0,0
6,d1WXoi0JgoNtrjLCO1xgRw,EgBYA6D6g4SrA6WNbbOtGg,ujHiaprwCQ5ewziu0Vi9rw,1,2017-04-30,"[go, buffet, around, pm, saturday, could, not,...",0,0,0
7,j1KJJv7K7LI9b3i6NDAT8A,CqdvpF3YXkWhEjL69qpPvQ,wk3D48R3Brvhci3gYcA7nw,5,2018-06-19,"[hand, best, hvac, experience, ever, team, inc...",1,1,1
8,vQBB1Ehm8jDvTlTjp7Ol3Q,5Lvl0F_0Q3PS54xi-0fURA,e8ZuGeCuJARJZnd4SERG8w,2,2014-08-25,"[mixed, review, regular, salon, close, sunday,...",0,0,0
9,ObH3YRz3GfR-8EPF4lxVEw,GiFcYCH6jgs--_Dwmk2JpA,-PJgXks184iZSA9ueAHe_A,1,2013-01-28,"[staff, friendly, visit, quick, get, bill, tho...",2,0,0


# Cell below creates a class that will be used in the vectorization code a few cells down. FYI, path in "_load_words" is machine specific. Also, you have to download the GloVe training set from GloVe website.

In [11]:

class MeanEmbeddingTransformer(TransformerMixin):
    
    def __init__(self):
        self._vocab, self._E = self._load_words()
        
    
    def _load_words(self):
        E = {}
        vocab = []

        with open(r'C:\Users\Mark\YelpProject\poop\glove.6B.50d.txt', encoding="utf8") as file:
            for i, line in enumerate(file):
                l = line.split(' ')
                if l[0].isalpha():
                    v = [float(i) for i in l[1:]]
                    E[l[0]] = numpy.array(v)
                    vocab.append(l[0])
        return numpy.array(vocab), E            

    
    def _get_word(self, v):
        for i, emb in enumerate(self._E):
            if numpy.array_equal(emb, v):
                return self._vocab[i]
        return None
    #in _doc_mean Ithink maybe we should get rid of w.lower.strip
    def _doc_mean(self, doc):
        return numpy.mean(numpy.array([self._E[w.lower().strip()] for w in doc if w.lower().strip() in self._E]), axis=0)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return numpy.array([self._doc_mean(doc) for doc in X])
    
    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)
    
    
#retrived from https://www.kaggle.com/nhrade/text-classification-using-word-embeddings

# Cell below is attempting to create a class for gensim doc2vec but it's not working yet and tbh I'm not sure how close I am

In [12]:
import os
from gensim.corpora import Dictionary
from gensim.matutils import sparse2full

class GensimDoc2Vec(TransformerMixin):
    def __init__(self, path=None):
        self.path = path
        self.id2word = None
        self.load()
    
    def load(self):
        if os.path.exists(self.path):
            self.id2word = Dictionary.load(self.path)
    
    def save(self):
        self.id2word.save(self.path)
    
    def fit(self, documents, labels = None):
        self.id2word = Dictionary(documents)
        self.save()
        return self
    
    def transform(self, documents):
        for document in documents:
            docvec = self.id2word.doc2vec
            yield sparse2full(docvec, len(self.id2word))
    

In [13]:
def GloVeMeanEmbedding(text):
    met = MeanEmbeddingTransformer()
    GloVeVector = numpy.array(met.fit_transform(map_on_column(text, _clean_review2)))
    return GloVeVector

In [14]:
def GenDoc2Vec(text, path):
    met = GensimDoc2Vec()
    GenVector = numpy.array(met.transform(map_on_column(text, _clean_review2)))
    return GenVector

# Cell below is vectorization. So far it can do TFIDF on specified number of n-grams, and I think the part with GloVe word embedding is almost functioning, but I haven't started "word2vec" vectorization yet.

In [15]:
def dummy_fun(doc):
    return doc

#def GloVeDistributedRep(text):
    ##GloVeVector = numpy.append(MeanEmbeddingTransformer().fit_transform(map_on_column(text, _clean_review2))
    #return GloVeVector


def vectorize(text, method, grams):
    if method == 'TFIDF':
        tfidf = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None, ngram_range = (1,grams))
        TDIFvectors = tfidf.fit_transform(map_on_column(text, _clean_review2))
        print(TDIFvectors)
    if method == 'DistributedRep':
        GloVevectors = GloVeDistributedRep(map_on_column(text, _clean_review2))
        print(GloVevectors)
    if method == 'Word2Vec':
        print('not yet defined')
    else:
        print('choose vectorization method')
        

def vectorize2(text, method, grams):
    if method == 'TFIDF':
        tfidf = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None, ngram_range = (1,grams))
        TDIFvectors = tfidf.fit_transform(text)
        print(TDIFvectors)
    if method == 'DistributedRep':
        GloVevectors = GloVeDistributedRep(text)
        print(GloVevectors)
    if method == 'Word2Vec':
        print('not yet defined')
    else:
        print('choose vectorization method')
        
        
        
def vectorize3(text, method, grams):
    if method == 'TFIDF':
        tfidf = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None, ngram_range = (1,grams))
        TDIFvectors = tfidf.fit_transform(text)
        print(TDIFvectors)
    if method == 'DistributedRep':
        GloVevectors = GloVeDistributedRep(text)
        print(GloVevectors)
    if method == 'Word2Vec':
        print('not yet defined')
    else:
        print('choose vectorization method')

In [16]:


import nltk.corpus
from nltk.text import TextCollection

def TFIDFvectorize(corpus):
    corpus = [_clean_review2(doc) for doc in corpus]
    texts = TextCollection(corpus)
    
    for doc in corpus:
        return {
            term: texts.tf_idf(term, doc)
            for term in doc
        }


# below is first attempt at pipeline. Just as a heads up, some models can't handel negative values and so can't be used with word embedding vectorization. Baysian, for example, can only handle positive vectors and so can be used with TF-IDF but not GloVe word embedding.

In [17]:


#the Vectorization_method arugment will be one of the classes that we wrote. That argument itself will have argumens within it.
def create_pipeline(documents, estimator, vectorization_method, ngrams, c=10000, reduction=False):
    steps = [
        ('vectorize', vectorization_method) 
    ]
    
    if reduction:
        steps.append((
        'reduction', TruncatedSVD(n_components=c)
        ))
    
    steps.append(('classifier', estimator))
    return Pipeline(steps)



In [18]:
#this line of code is basically just filler to make sure the pipeline is being used correct. It's functioning output should just be a list of the pipline steps.
create_pipeline(review_sample['review_text'], MultinomialNB, MeanEmbeddingTransformer(), 1)

Pipeline(memory=None,
     steps=[('vectorize', <__main__.MeanEmbeddingTransformer object at 0x0000025DF4D27358>), ('classifier', <class 'sklearn.naive_bayes.MultinomialNB'>)])

In [56]:
#rn the models are just using the default hyperparameters (for exxample, n_neighbors = 5. I'm not sure how to change this. I get an error when I try to change it here.
models = []
for form in (SGDClassifier, KNeighborsClassifier, LogisticRegression, LinearSVC, RandomForestClassifier):
    models.append(create_pipeline(review_sample['review_text'],form(), MeanEmbeddingTransformer(), 1))

In [20]:
for model in models:
    model.fit(review_sample['review_text'], review_sample['stars'])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [57]:
#really annoyting but rn the printout for this is pretty nasty. 

for model in models:
    scores = []
    model.fit(review_sample['review_text'], review_sample['stars'])
    y_pred = model.predict(review_sample['review_text'])
    score = accuracy_score(review_sample['stars'], y_pred)
    scores.append(score)
    print(model)
    print ("Accuracy of {} is {:03f}".format(model, numpy.mean(scores)))
    print(classification_report(review_sample['stars'], y_pred, labels= [1,2,3,4,5]))


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vectorize', <__main__.MeanEmbeddingTransformer object at 0x0000025DB45F5E10>), ('classifier', SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', ma...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])
Accuracy of Pipeline(memory=None,
     steps=[('vectorize', <__main__.MeanEmbeddingTransformer object at 0x0000025DB45F5E10>), ('classifier', SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', ma...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]) is 0.403000
              precision    recall  f1-score   support

           1       0.84      0.13      0.22       125
   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
     steps=[('vectorize', <__main__.MeanEmbeddingTransformer object at 0x0000025D956EC7B8>), ('classifier', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])
Accuracy of Pipeline(memory=None,
     steps=[('vectorize', <__main__.MeanEmbeddingTransformer object at 0x0000025D956EC7B8>), ('classifier', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]) is 0.578000
              precision    recall  f1-score   support

           1       0.63      0.70      0.66       125
           2       0.00      0.00      0.00        73
           3       

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Pipeline(memory=None,
     steps=[('vectorize', <__main__.MeanEmbeddingTransformer object at 0x0000025DA4DAB208>), ('classifier', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])
Accuracy of Pipeline(memory=None,
     steps=[('vectorize', <__main__.MeanEmbeddingTransformer object at 0x0000025DA4DAB208>), ('classifier', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]) is 0.588000
              precision    recall  f1-score   support

           1       0.57      0.74      0.65       125
           2       0.50      0.07      0.12        73
           3       0.51      0.19      0.28       124
           4       0.48      0.28      0.36       225
         

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('vectorize', <__main__.MeanEmbeddingTransformer object at 0x0000025DCB7DC2B0>), ('classifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])
Accuracy of Pipeline(memory=None,
     steps=[('vectorize', <__main__.MeanEmbeddingTransformer object at 0x0000025DCB7DC2B0>), ('classifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]) is 0.991000
              precision    recall  f1-score   support

           1       0.99      0.98      0.99       125
   

In [58]:
models3 = []
for form in (MultinomialNB, SGDClassifier, KNeighborsClassifier, LogisticRegression, LinearSVC, RandomForestClassifier):
    models3.append(create_pipeline(review_sample['review_text'],form(), TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun,token_pattern=None, ngram_range=(1,2)), 1))
    

In [59]:
for model in models3:
    scores = []
    model.fit(review_sample['review_text'], review_sample['stars'])
    y_pred = model.predict(review_sample['review_text'])
    score = accuracy_score(review_sample['stars'], y_pred)
    scores.append(score)
    print ("Accuracy of {} is {:03f}".format(model, numpy.mean(scores)))
    y_pred = model.predict(review_sample['review_text'])
    print(classification_report(review_sample['stars'], y_pred, labels= [1,2,3,4,5]))

Accuracy of Pipeline(memory=None,
     steps=[('vectorize', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2',
        preprocessor=<function ...      vocabulary=None)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))]) is 0.490000
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       125
           2       0.00      0.00      0.00        73
           3       0.00      0.00      0.00       124
           4       1.00      0.16      0.28       225
           5       0.47      1.00      0.64       453

   micro avg       0.49      0.49      0.49      1000
   macro avg       0.29      0.23      0.18      1000
weighted avg       0.44      0.49      0.35      1000



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


Accuracy of Pipeline(memory=None,
     steps=[('vectorize', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2',
        preprocessor=<function ...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]) is 1.000000
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       125
           2       1.00      1.00      1.00        73
           3       1.00      1.00      1.00       124
           4       1.00      1.00      1.00       225
           5       1.00      1.00      1.00       453

   micro avg       1.00      1.00      1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

Accuracy of Pipeline(memory=None,
     steps=[('vectori

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy of Pipeline(memory=None,
     steps=[('vectorize', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2',
        preprocessor=<function ...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]) is 0.640000
              precision    recall  f1-score   support

           1       1.00      0.38      0.55       125
           2       0.00      0.00      0.00        73
           3       1.00      0.15      0.25       124
           4       0.99      0.54      0.70       225
           5       0.56      1.00      0.72       453

   micro avg       0.64      0.64      0.64      1000
   macro avg       0.71      0.41      0.44      1000
weighted avg       0.72      0.64      0.58      1000



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Accuracy of Pipeline(memory=None,
     steps=[('vectorize', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2',
        preprocessor=<function ...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]) is 1.000000
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       125
           2       1.00      1.00      1.00        73
           3       1.00      1.00      1.00       124
           4       1.00      1.00      1.00       225
           5       1.00      1.00      1.00       453

   micro avg       1.00      1.00      1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy of Pipeline(memory=None,
     steps=[('vectorize', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2',
        preprocessor=<function ...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]) is 0.983000
              precision    recall  f1-score   support

           1       1.00      0.99      1.00       125
           2       0.99      0.97      0.98        73
           3       1.00      0.99      1.00       124
           4       1.00      0.95      0.97       225
           5       0.97      1.00      0.98       453

   micro avg       0.98      0.98      0.98      1000
   macro avg       0.99      0.98      0.98      1000
weighted avg       0.98      0.98      0.98      1000



# Below is a visualization of our data. I was hoping that some visual clusters would pop out but no luck. If you get a  n_components must be < n_features; got 50 >= 50 error then set n_components less than 50 in TSNE visualizer.

In [ ]:
from yellowbrick.text import TSNEVisualizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the hobbies corpus (don't need to do again, included for completeness)

vectorizor = MeanEmbeddingTransformer()
vectorizor1 = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None, ngram_range = (1,2))

docs = vectorizor.fit_transform(review_sample['review_text'])
labels = review_sample['stars']

# Create the visualizer and draw the vectors
tsne = TSNEVisualizer(size=(1080, 720))
tsne.fit(docs, labels)
# Note: TSNE is a model visualizer not a feature visualizer, 
# so it has no transform method!
tsne.poof()